# 1. Configuração

## 1.1 Instalar dependências

In [ ]:
%pip install transformers datasets peft accelerate bitsandbytes xformers unsloth

## 1.2 Baixar dataset

In [ ]:
!wget -O "LF-Amazon-1.3M.zip" "https://drive.usercontent.google.com/download?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK&export=download&authuser=0&confirm=t"
!unzip "LF-Amazon-1.3M.zip"
!gzip --decompress "./LF-Amazon-1.3M/trn.json.gz"
!mv "./LF-Amazon-1.3M/trn.json" "./trn.json"


## 1.3 Imports

In [ ]:
import json
import pandas as pd
from datasets import Dataset
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)

## 1.4 Verificar se possui CUDA

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Treinando no dispositivo: {device}")

# 2. Inicialização

In [ ]:
file_path = "trn.json"

data = []
with open(file_path, "r") as file:
    for line in file:
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Erro ao decodificar linha: {line}")
            print(f"Detalhes do erro: {e}")

filtered_data = [{"title": entry["title"], "content": entry["content"]} for entry in data]

output_file_path = "filtered_data.json"
with open(output_file_path, "w") as output_file:
    json.dump(filtered_data, output_file, indent=4)

print(f"Arquivo filtrado salvo em: {output_file_path}")

In [ ]:
# Liberar RAM
del data
del filtered_data

In [ ]:
# Carregar o arquivo JSON
with open('filtered_data.json', 'r') as f:
    data = json.load(f)

# Converter para DataFrame
df = pd.DataFrame(data)

In [ ]:
def create_prompt(row):
    return {
        "input": f"You are a book expert and should answer any question involving this title {row['title']}",
        "output": row["content"]
    }

# Aplicar ao DataFrame
fine_tune_data = df.apply(create_prompt, axis=1)
fine_tune_data

In [ ]:
# Dividir os dados em treinamento e validação
from sklearn.model_selection import train_test_split

# Separar 10% dos dados para validação
train_data, val_data = train_test_split(fine_tune_data, test_size=0.1)

In [ ]:
# Liberar RAM
del fine_tune_data

In [ ]:
with open('train_data.json', 'w') as f:
    train_data.to_json('train_data.json', orient='records', lines=True)

with open('val_data.json', 'w') as f:
    val_data.to_json('val_data.json', orient='records', lines=True)

In [ ]:
# Liberar RAM
del train_data
del val_data
del df

# 3. Treino

In [ ]:
# Carregar os datasets
train_dataset = Dataset.from_json("train_data.json")
val_dataset = Dataset.from_json("val_data.json")


In [ ]:
# Escolha do modelo
model_name = "t5-small"  # Altere para o modelo desejado, como "mistralai/Mistral-7B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Mover o modelo para o dispositivo
model = model.to(device)  # Modificação: Move o modelo para GPU ou CPU
print(f"Modelo carregado no dispositivo: {model.device}")

# Função de tokenização
def tokenize_function(examples):
    return tokenizer(
        examples["input"],
        text_target=examples["output"],
        truncation=True,
        padding="max_length",
        max_length=256  # Modificação: Reduz o comprimento máximo para economizar memória
    )

# Tokenizar os datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

In [ ]:
# Liberar RAM
del train_dataset
del val_dataset

In [ ]:
# Save
tokenized_train_dataset.save_to_disk("tokenized_train_dataset")
tokenized_val_dataset.save_to_disk("tokenized_val_dataset")

In [ ]:
# Read from saved tokenized
tokenized_train_dataset = Dataset.load_from_disk("tokenized_train_dataset")
tokenized_val_dataset = Dataset.load_from_disk("tokenized_val_dataset")

In [ ]:
# Configurações de treinamento
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=2,  # Modificação: Reduz o batch size para evitar OOM
    gradient_accumulation_steps=4,  # Modificação: Acumula gradientes para simular batches maiores
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    fp16=True,  # Modificação: Habilitar Mixed Precision Training
    logging_dir='./logs',
    predict_with_generate=True,
)



In [ ]:
# Criar o Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
# Save trainer
trainer.save_model("./results")

In [ ]:
# Load trainer
trainer = Seq2SeqTrainer.from_pretrained("./results")

# 4. Uso

In [ ]:
# Função para perguntas
def ask_question(question, model, tokenizer, device):
    inputs = tokenizer(question, return_tensors="pt")
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Modificação: Move os dados para o dispositivo correto
    outputs = model.generate(
    **inputs,
    max_length=512,  # Corrigido: max_length
    num_beams=10,  # Beam search para respostas melhores
    no_repeat_ngram_size=2,  # Evitar repetições
    temperature=0.7,  # Controlar aleatoriedade
    top_k=50,  # Filtrar os 50 tokens mais prováveis
    top_p=0.95,  # Top-p sampling para diversidade
    do_sample=True
)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# Exemplo de teste
question = input("Pergunta: ") # "You are a book expert and should answer any question involving this title Collins German Unabridged Dictionary 5th Edition (Harpercollins Unabridged Dictionaries)"
response = ask_question(question, model, tokenizer, device)
print("Pergunta:", question)
print("Resposta:", response)